In [131]:
import pymysql as mysql

server = "sql7.freemysqlhosting.net"
user = "sql7271447"
password = "hi7DSvMiPz"
database = "sql7271447"

Данные будем использовать с сайта MyAnimeList.

А именно информацию пользователей и информацию о тайтлах. 


In [107]:
anime, users = pd.read_csv('anime_cleaned.csv'), pd.read_csv('UserList.csv')

In [108]:
anime.head(4)

,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,...,broadcast,related,producer,licensor,studio,genre,opening_theme,ending_theme,duration_min,aired_from_year
0,11013,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,Youko x Boku SS,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,12,Finished Airing,...,Fridays at Unknown,"{'Adaptation': [{'mal_id': 17207, 'type': 'man...","Aniplex, Square Enix, Mainichi Broadcasting Sy...",Sentai Filmworks,David Production,"Comedy, Supernatural, Romance, Shounen","['""Nirvana"" by MUCC']","['#1: ""Nirvana"" by MUCC (eps 1, 11-12)', '#2: ...",24.0,2012.0
1,2104,Seto no Hanayome,My Bride is a Mermaid,瀬戸の花嫁,The Inland Sea Bride,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,26,Finished Airing,...,Unknown,"{'Adaptation': [{'mal_id': 759, 'type': 'manga...","TV Tokyo, AIC, Square Enix, Sotsu",Funimation,Gonzo,"Comedy, Parody, Romance, School, Shounen","['""Romantic summer"" by SUN&LUNAR']","['#1: ""Ashita e no Hikari (明日への光)"" by Asuka Hi...",24.0,2007.0
2,5262,Shugo Chara!! Doki,Shugo Chara!! Doki,しゅごキャラ！！どきっ,"Shugo Chara Ninenme, Shugo Chara! Second Year",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,51,Finished Airing,...,Unknown,"{'Adaptation': [{'mal_id': 101, 'type': 'manga...","TV Tokyo, Sotsu",NaN,Satelight,"Comedy, Magic, School, Shoujo","['#1: ""Minna no Tamago (みんなのたまご)"" by Shugo Cha...","['#1: ""Rottara Rottara (ロッタラ ロッタラ)"" by Buono! ...",24.0,2008.0
3,721,Princess Tutu,Princess Tutu,プリンセスチュチュ,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,38,Finished Airing,...,Fridays at Unknown,"{'Adaptation': [{'mal_id': 1581, 'type': 'mang...","Memory-Tech, GANSIS, Marvelous AQL",ADV Films,Hal Film Maker,"Comedy, Drama, Magic, Romance, Fantasy","['""Morning Grace"" by Ritsuko Okazaki']","['""Watashi No Ai Wa Chiisaikeredo"" by Ritsuko ...",16.0,2002.0


In [109]:
users.head(4)

,username,user_id,user_watching,user_completed,user_onhold,user_dropped,user_plantowatch,user_days_spent_watching,gender,location,birth_date,access_rank,join_date,last_online,stats_mean_score,stats_rewatched,stats_episodes
0,karthiga,2255153,3,49,1,0,0,55.31,Female,"Chennai, India",1990-04-29,NaN,2013-03-03,2014-02-04 01:32:00,7.43,0.0,3391.0
1,RedvelvetDaisuki,1897606,61,396,39,0,206,118.07,Female,Manila,1995-01-01,NaN,2012-12-13,1900-05-13 02:47:00,6.78,80.0,7094.0
2,Damonashu,37326,45,195,27,25,59,83.70,Male,"Detroit,Michigan",1991-08-01,NaN,2008-02-13,1900-03-24 12:48:00,6.15,6.0,4936.0
3,bskai,228342,25,414,2,5,11,167.16,Male,"Nayarit, Mexico",1990-12-14,NaN,2009-08-31,2014-05-12 16:35:00,8.27,1.0,10081.0


Посколь данные получены с помощью обкачки HTML страниц, в базе данных хранить их все нет необходимости.
Также для наглядности, не будем использовать все данные, а возьмем первых 100 тайтлов и 10 пользователей, чтобы было несложно придумать информацию для них.

In [110]:
anime, users = anime.dropna()[:100], users.fillna('')[:15]

In [111]:
anime = anime[['title',
               'episodes',
               'type',
               'status',
               'studio',
               'genre',
               'duration',
               'rating',
               'title_synonyms']]

users = users[['username',
               'gender',
               'location',
               'join_date']]

In [112]:
anime_list, users_list = anime.values, users.values

________

In [113]:
def add(table, columns, values):
    db = mysql.connect(server,
                       user,
                       password,
                       database)
    with db.cursor() as cursor:
        sql = "INSERT INTO {}({}) VALUES ({})"
        cursor.execute(sql.format(table,
                                  ", ".join(columns),
                                  "'" + str("', '".join(values)) + "'"))
    db.commit()
    db.close()

Начнем с добавления пользователей. Придумаем им самый просто e-mail, а также для некоторых заполним поле информации.

In [114]:
mails = [str(user) + '@pochta.ru' for user in users['username']]
info = ['',
        'Hope we will find anything to talk about and recommend each other.',
        '',
        '',
        "I am ordinary programmer guy",
        "My tastes are quite specific, but apparently understandable" ,
        '',
        '',
        "My peers do not do as much watching as I",
        'I like to consider myself a real thinker',
        'but rather you will hear references to Code Geass',
        '',
        '',
        '',
        'have a talk becoming of such a status — either way']

In [132]:
%%time

for Username, Mail, Joined, Gender, Location, Info in zip(users['username'],
                                                          mails,
                                                          users['join_date'],
                                                          users['gender'],
                                                          users['location'],
                                                          info):
    add('Users',
        ['Username', 'Mail', 'Joined', 'Gender', 'Location', 'Info'],
        [str(Username), str(Mail), str(Joined), str(Gender), str(Location), str(Info)])

CPU times: user 44.7 ms, sys: 17.2 ms, total: 61.9 ms
Wall time: 8.82 s


In [116]:
def look_something(table, thing):
    db = mysql.connect(server, user, password, database)
    with db.cursor() as cursor:
        sql = "SELECT {} FROM {}"
        cursor.execute(sql.format(thing, table))
        items = cursor.fetchall()
    db.commit()
    db.close()
    return items

In [125]:
for r in look_something('Users', '*'):
    print(r)

(1, 'karthiga', 'karthiga@pochta.ru', datetime.date(2013, 3, 3), 'Female', 'Chennai, India ', '')
(2, 'RedvelvetDaisuki', 'RedvelvetDaisuki@pochta.ru', datetime.date(2012, 12, 13), 'Female', 'Manila', 'Hope we will find anything to talk about and recommend each other.')
(3, 'Damonashu', 'Damonashu@pochta.ru', datetime.date(2008, 2, 13), 'Male', 'Detroit,Michigan', '')
(4, 'bskai', 'bskai@pochta.ru', datetime.date(2009, 8, 31), 'Male', 'Nayarit, Mexico', '')
(5, 'shuzzable', 'shuzzable@pochta.ru', datetime.date(2013, 3, 25), '', '', 'I am ordinary programmer guy')
(6, 'terune_uzumaki', 'terune_uzumaki@pochta.ru', datetime.date(2010, 5, 10), 'Female', 'Malaysia, Kuantan', 'My tastes are quite specific, but apparently understandable')
(7, 'Bas_G', 'Bas_G@pochta.ru', datetime.date(2015, 11, 26), 'Male', 'Nijmegen, Nederland', '')
(8, 'punane', 'punane@pochta.ru', datetime.date(2008, 2, 24), '', '', '')
(9, '-otaku-sama-', '-otaku-sama-@pochta.ru', datetime.date(2015, 4, 27), '', '', 'My pe

У нас появились пользователи, с которыми мы продолжим работу немного позже. Теперь необходимо добавить сами тайтлы, которые люди будут добавлять к себе в хранилище

In [118]:
anime.head(2)

,title,episodes,type,status,studio,genre,duration,rating,title_synonyms
0,Inu x Boku SS,12,TV,Finished Airing,David Production,"Comedy, Supernatural, Romance, Shounen",24 min. per ep.,PG-13 - Teens 13 or older,Youko x Boku SS
12,Sekaiichi Hatsukoi 2,12,TV,Finished Airing,Studio Deen,"Comedy, Drama, Romance, Shounen Ai",24 min. per ep.,PG-13 - Teens 13 or older,"Sekai-ichi Hatsukoi 2, Sekai&#039;ichi Hatsukoi 2"


In [133]:
%%time

for item in anime.values:
    add('Films',
        ['Title',
         'Episodes',
         'Type',
         'Status',
         'Studios',
         'Genres',
         'Duration',
         'Rating',
         'Synonyms'],
        [str(item[0]),
         str(item[1]),
         str(item[2]),
         str(item[3]),
         str(item[4]),
         str(item[5]),
         str(item[6]),
         str(item[7]),
         str(item[8])])

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pymysql/cursors.py:170: Warning: (1265, "Data truncated for column 'Genres' at row 1")
  result = self._query(query)


CPU times: user 315 ms, sys: 122 ms, total: 437 ms
Wall time: 1min 21s


In [134]:
for r in look_something('Films', '*'):
    print(r)

(1, 'Inu x Boku SS', 12, 'TV', 'Finished Airing', 'David Production', 'Comedy, Supernatural, Romance, Shounen', 24, 'PG-13', 'Youko x Boku SS')
(2, 'Sekaiichi Hatsukoi 2', 12, 'TV', 'Finished Airing', 'Studio Deen', 'Comedy, Drama, Romance, Shounen Ai', 24, 'PG-13', 'Sekai-ichi Hatsukoi 2, Sekai&#039;ichi Hatsukoi 2')
(3, 'Ranma ½', 161, 'TV', 'Finished Airing', 'Studio Deen', 'Slice of Life, Comedy, Martial Arts, Fantasy', 24, 'R+', 'Ranma 1/2, Ranma ½ Nettou Hen')
(4, 'Toradora!', 25, 'TV', 'Finished Airing', 'J.C.Staff', 'Slice of Life, Comedy, Romance, School', 24, 'PG-13', 'Tiger X Dragon')
(5, 'Chihayafuru', 25, 'TV', 'Finished Airing', 'Madhouse', 'Drama, Game, Josei, School, Slice of Life, Sports', 22, 'PG-13', 'Chihayafull')
(6, 'Itazura na Kiss', 25, 'TV', 'Finished Airing', 'TMS Entertainment', 'Comedy, Romance, Shoujo', 24, 'PG-13', 'Naughty Kiss, Teasing Kiss, Mischievous Kiss, Itazurana Kiss, It Started With A Kiss')
(7, 'Fruits Basket', 26, 'TV', 'Finished Airing', 'Stud

Заметим, что длительность имеет один формат и нам хорошо бы привести это поле к числовому значению. Также можно преобразовать возрастное ограничение, чтобы было лугчи с этим работать. Все остально похоже на правду и с этим можно работать. 

In [128]:
def new_duration(line):
    return line.split(' ')[0]

def new_rate(line):
    return line.split(' - ')[0]

In [129]:
anime['duration'] = anime['duration'].apply(new_duration)
anime['rating'] = anime['rating'].apply(new_rate)

In [130]:
for r in look_something('Films', '*'):
    print(r)

(1, 'Inu x Boku SS', 12, 'TV', 'Finished Airing', 'David Production', 'Comedy, Supernatural, Romance, Shounen', 24, 'PG-13', 'Youko x Boku SS')
(2, 'Sekaiichi Hatsukoi 2', 12, 'TV', 'Finished Airing', 'Studio Deen', 'Comedy, Drama, Romance, Shounen Ai', 24, 'PG-13', 'Sekai-ichi Hatsukoi 2, Sekai&#039;ichi Hatsukoi 2')
(3, 'Ranma ½', 161, 'TV', 'Finished Airing', 'Studio Deen', 'Slice of Life, Comedy, Martial Arts, Fantasy', 24, 'R+', 'Ranma 1/2, Ranma ½ Nettou Hen')
(4, 'Toradora!', 25, 'TV', 'Finished Airing', 'J.C.Staff', 'Slice of Life, Comedy, Romance, School', 24, 'PG-13', 'Tiger X Dragon')
(5, 'Chihayafuru', 25, 'TV', 'Finished Airing', 'Madhouse', 'Drama, Game, Josei, School, Slice of Life, Sports', 22, 'PG-13', 'Chihayafull')
(6, 'Itazura na Kiss', 25, 'TV', 'Finished Airing', 'TMS Entertainment', 'Comedy, Romance, Shoujo', 24, 'PG-13', 'Naughty Kiss, Teasing Kiss, Mischievous Kiss, Itazurana Kiss, It Started With A Kiss')
(7, 'Fruits Basket', 26, 'TV', 'Finished Airing', 'Stud

Теперь у нас своя небольшая база данных с аниме и пользователями. Дальше, в файле Functions.ipynb мы реализуем таблицу для хранения просмотров, рецензий и небольшого форума (дополнительно)